In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import pandas as pd

****Heterologous expression of 5-HTP and MH4 pathway in Bacillus subtilis****

In [2]:
model = read_sbml_model('data/iYO844.xml')

In [3]:
model

Name,iYO844
Memory address,0x07ff44267a190
Number of metabolites,990
Number of reactions,1250
Number of groups,0
Objective expression,1.0*BIOMASS_BS_10 - 1.0*BIOMASS_BS_10_reverse_8788b
Compartments,"cytosol, extracellular space"


In [4]:
for metabolite in model.metabolites.query('Formate', 'name'):
    print(metabolite.name, metabolite.id)

Formate for_c
Formate for_e


In [5]:
for metabolite in model.metabolites.query('GTP', 'name'):
    print(metabolite.name, metabolite.id)

DGTP C10H12N5O13P3 dgtp_c
GTP C10H12N5O14P3 gtp_c


In [6]:
for metabolite in model.metabolites.query('H2O', 'name'):
    print(metabolite.name, metabolite.id)

H2O H2O h2o_c
H2O H2O h2o_e
Oxaloglycolate C4H2O6 oxglyc_c
Sulfoacetate C2H2O5S sula_c
Sulfoacetate C2H2O5S sula_e


In [7]:
for metabolite in model.metabolites.query('Inorganic triphosphate', 'name'):
    print(metabolite.name, metabolite.id, metabolite.formula)

Inorganic triphosphate pppi_c HO10P3


In [8]:
for metabolite in model.metabolites.query('Nicotinamide', 'name'):
    print(metabolite.name, metabolite.id)

Nicotinamide adenine dinucleotide nad_c
Nicotinamide adenine dinucleotide - reduced nadh_c
Nicotinamide adenine dinucleotide phosphate nadp_c
Nicotinamide adenine dinucleotide phosphate - reduced nadph_c
Nicotinamide ncam_c


In [9]:
model.metabolites.h_c 

Metabolite identifier,h_c
Name,H+
Memory address,0x07ff4939ed9d0
Formula,H
Compartment,c
In 616 reaction(s),"ARGt2r, DPR, ADNK1, MOPSabc, CYTDK2, METSabc, ABUTD, LLEUDr, ETHAt6, ARSENBabc, 3UMPt6, LAAL29, L_LACt2r, ACPpds, ADSK, XAND, ALAt2r, DADNt2, GLCURt2r, IPDPS, PPS, LCTSabc, G1PACT, LAAL2, Kt3r,..."


In [10]:
for metabolite in model.metabolites.query('pterin', 'name'):
    print(metabolite.name, metabolite.id, metabolite.formula)

6-Pyruvoyl-5,6,7,8-tetrahydropterin 6pthp_c C9H11N5O3
Dihydroneopterin dhnpt_c C9H13N5O4


In [11]:
for metabolite in model.metabolites.query('Tryptophan', 'name'):
    print(metabolite.name, metabolite.id)

L-Tryptophan trp__L_c
L-Tryptophan trp__L_e


In [12]:
for metabolite in model.metabolites.query('O2 O2', 'name'):
    print(metabolite.name, metabolite.id)

O2 O2 o2_c
O2 O2 o2_e


In [13]:
MH2_3P = Metabolite(id='MH2_3P', compartment='c')

In [14]:
preMH2_3P = Metabolite(id='preMH2_3P', compartment='c')

In [15]:
MH2 = Metabolite(id='MH2', compartment='c')

In [16]:
MH4 = Metabolite(id='MH4', compartment='c')

In [17]:
HydroxyMH4 = Metabolite(id='hydroxy_MH4', compartment='c')

In [18]:
hydroxy_trp = Metabolite (id='hydroxy_trp', compartment='c')

In [19]:
folE = Reaction('R05048')
folE.add_metabolites({model.metabolites.h2o_c: -1,
                      model.metabolites.gtp_c: -1,
                      model.metabolites.for_c: 1,
                      preMH2_3P: 1
                     })
print(folE.build_reaction_string())

gtp_c + h2o_c --> for_c + preMH2_3P


In [20]:
folX = Reaction('R11082')
folX.add_metabolites({preMH2_3P: -1,
                      MH2_3P: 1
                     })
print(folX.build_reaction_string())

preMH2_3P --> MH2_3P


In [21]:
MH2_3P_dephosphorylation = Reaction ('Putative_dephosphorylation')
MH2_3P_dephosphorylation.add_metabolites({MH2_3P: -1,
                                          MH2: 1,
                                          model.metabolites.pppi_c: 1
                                         })
print(MH2_3P_dephosphorylation.build_reaction_string())

MH2_3P --> MH2 + pppi_c


In [22]:
folM = Reaction('R11019')
folM.add_metabolites ({model.metabolites.nadph_c: -1,
                       model.metabolites.h_c: -1,
                       MH2: -1,
                       MH4: 1,
                       model.metabolites.nadp_c: 1
                      })
print(folM.build_reaction_string())

MH2 + h_c + nadph_c --> MH4 + nadp_c


In [23]:
phhB = Reaction ('R04734')
phhB.add_metabolites({HydroxyMH4: -1,
                      MH2: 1,
                      model.metabolites.h2o_c: 1
                     })
print(phhB.build_reaction_string())

hydroxy_MH4 --> MH2 + h2o_c


In [24]:
phhA = Reaction ('R12540')
phhA.add_metabolites({model.metabolites.trp__L_c: -1,
                      model.metabolites.o2_c: -1,
                      MH4: -1,
                      HydroxyMH4: 1,
                      hydroxy_trp: 1
                     })
print(phhA.build_reaction_string())

MH4 + o2_c + trp__L_c --> hydroxy_MH4 + hydroxy_trp


In [25]:
model.add_reactions([folE])

In [26]:
model.add_reactions([folX])

In [27]:
model.add_reactions([MH2_3P_dephosphorylation])

In [28]:
model.add_reactions([folM])

In [29]:
model.add_reactions([phhB])

In [30]:
model.add_reactions([phhA])

In [31]:
model.reactions.R12540

Reaction identifier,R12540
Name,
Memory address,0x07ff49308e550
Stoichiometry,MH4 + o2_c + trp__L_c --> hydroxy_MH4 + hydroxy_trp + O2 O2 + L-Tryptophan --> +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [32]:
hydroxy_5HTP_exchange = Reaction('EX_hydroxy_trp')
hydroxy_5HTP_exchange.add_metabolites({model.metabolites.hydroxy_trp: -1})
model.add_reactions([hydroxy_5HTP_exchange])

In [33]:
MH4_exchange = Reaction('EX_MH4')
MH4_exchange.add_metabolites({model.metabolites.MH4: -1})
model.add_reactions([MH4_exchange])

In [34]:
model.optimize()

,fluxes,reduced_costs
EX_etha_e,0.0,-0.066773
EX_drib_e,0.0,-0.150240
EX_csn_e,0.0,-0.129837
EX_5mtr_e,0.0,-0.224433
EX_dtmp_e,0.0,-0.337577
...,...,...
R11019,0.0,0.000000
R04734,0.0,-0.012984
R12540,0.0,-0.391366
EX_hydroxy_trp,0.0,0.000000


In [35]:
with model:
    print(model.reactions.R12540)
    model.objective = model.reactions.R12540
    max_5HTP_production = model.optimize().objective_value
    print(max_5HTP_production)

R12540: MH4 + o2_c + trp__L_c --> hydroxy_MH4 + hydroxy_trp
0.5776145203111468


In [36]:
from cobra.flux_analysis import flux_variability_analysis

In [37]:
flux_variability_analysis(model)

,minimum,maximum
EX_etha_e,0.0,0.000000e+00
EX_drib_e,0.0,0.000000e+00
EX_csn_e,0.0,0.000000e+00
EX_5mtr_e,0.0,5.621751e-14
EX_dtmp_e,0.0,1.016240e-13
...,...,...
R11019,0.0,1.062855e-13
R04734,0.0,8.483340e-14
R12540,0.0,8.483340e-14
EX_hydroxy_trp,0.0,8.483340e-14


In [38]:
hydroroxy_trp_flux_analysis = flux_variability_analysis(model, model.reactions.EX_hydroxy_trp, fraction_of_optimum=0.1)
print(hydroroxy_trp_flux_analysis)

                minimum   maximum
EX_hydroxy_trp      0.0  0.520739


In [56]:
for gene in model.genes:
    with model:
        gene.knock_out()
        model.slim_optimize()
        hydroxy_trp_flux = model.reactions.EX_hydroxy_trp.flux
        model.objective_value = model.reactions.EX_hydroxy_trp
        model.slim_optimize()
        if hydroxy_trp_flux != 0:  print(model.genes.get_by_id(gene.id), hydroxy_trp_flux, model.objective_value.flux)

BSU00760 0.25862068965517115 0.25862068965517115
BSU00740 0.26904465397616084 0.26904465397616084
BSU36900 0.28622261174408187 0.28622261174408187
BSU06520 0.5513618677039211 0.5513618677039211
BSU06420 0.23288946162644344 0.23288946162644344
BSU06430 0.23288946162644344 0.23288946162644344
BSU21810 0.1668520952410034 0.1668520952410034
BSU28080 0.16685209523945135 0.16685209523945135
BSU00780 0.23288946162657415 0.23288946162657415
BSU15540 0.23288946162657406 0.23288946162657406
BSU00770 0.25696887686062236 0.25696887686062236
BSU06490 0.2862226117440774 0.2862226117440774
BSU06360 0.2673431056306397 0.2673431056306397
BSU22780 0.23288946162657403 0.23288946162657403
BSU22740 5.596706339097297e-17 5.596706339097297e-17
BSU22760 5.596706339097297e-17 5.596706339097297e-17
BSU22370 0.23841059602648715 0.23841059602648715
BSU00790 0.2586206896551724 0.2586206896551724
BSU27850 0.16216372998280443 0.16216372998280443
BSU22730 0.16542044739576883 0.16542044739576883
BSU35720 0.16542044693